### Implements algorithm 1 in the parti-game [paper](https://link.springer.com/content/pdf/10.1007/BF00993591.pdf)

Learn a controller from a start region to a goal region on a continuous space; Four increasingly effective algorithms to partition discrete state spaces. Algorithm 1 and 2 are non-learning; 3 and 4 learn, hence explore, the world while planning a route to the goal. Here, I implement algorithm 1

In [2]:
import sys
import numpy as np
from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('..', 'grids')))
sys.path.append(abspath(join('..', 'utils')))
sys.path.append(abspath(join('..')))

from create_grid import createGrid
from process_grid import processGrid
import matplotlib.pyplot as plt
from gen_utils import *

In [3]:
import numpy as np
from math import pi
from utils import expand, zeros, Bundle
from grids import createGrid
from valFuncs import proj
from visualization import visSetIm
import matplotlib.pyplot as plt

import logging
logger = logging.getLogger(__name__)

def tutorial():
    """
      Reproduces Sylvia's Tutorial on BRS
         1. Run Backward Reachable Set (BRS) with a goal
             uMode = 'min' <-- goal
             minWith = 'none' <-- Set (not tube)
             compTraj = false <-- no trajectory
         2. Run BRS with goal, then optimal trajectory
             uMode = 'min' <-- goal
             minWith = 'none' <-- Set (not tube)
             compTraj = True <-- compute optimal trajectory
         3. Run Backward Reachable Tube (BRT) with a goal, then optimal trajectory
             uMode = 'min' <-- goal
             minWith = 'minVOverTime' <-- Tube (not set)
             compTraj = True <-- compute optimal trajectory
         4. Add disturbance
             dStep1: define a dMax (dMax = [.25, .25, 0];)
             dStep2: define a dMode (opposite of uMode)
             dStep3: input dMax when creating your DubinsCar
             dStep4: add dMode to schemeData
         5. Change to an avoid BRT rather than a goal BRT
             uMode = 'max' <-- avoid
             dMode = 'min' <-- opposite of uMode
             minWith = 'minVOverTime' <-- Tube (not set)
             compTraj = false <-- no trajectory
         6. Change to a Forward Reachable Tube (FRT)
             add schemeData.tMode = 'forward'
             note: now having uMode = 'max' essentially says "see how far I can
             reach"
         7. Add obstacles
             add the following code:
             obstacles = shapeCylinder(g, 3, [-1.5; 1.5; 0], 0.75);
             HJIextraArgs.obstacles = obstacles;
         8. Add random disturbance (white noise)
             add the following code:
             HJIextraArgs.addGaussianNoiseStandardDeviation = [0; 0; 0.5];
    """

    ## Should we compute the trajectory?
    compTraj = True;

    ## Grid
    grid_min = expand(np.array((-5, -5, -pi)), ax = 1); # Lower corner of computation domain
    grid_max = expand(np.array((5, 5, -pi)), ax = 1);   # Upper corner of computation domain
    N = expand(np.array((41, 41,  41)), ax = 1);        # Number of grid points per dimension
    pdDims = 3;               # 3rd dimension is periodic
    g = createGrid(grid_min, grid_max, N, pdDims);
    # Use "g = createGrid(grid_min, grid_max, N);" if there are no periodic
    # state space dimensions

    ## target set
    R = 1;
    # data0 = shapeCylinder(grid,ignoreDims,center,radius)
    data0 = shapeCylinder(g, 3, zeros(3, 1), R);
    # also try shapeRectangleByCorners, shapeSphere, etc.

    ## time vector
    t0 = 0;
    tMax = 2;
    dt = 0.05;
    tau = np.arange(t0, tMax+dt, dt); # account for pythonb's 0-indexing

    ## problem parameters

    # input bounds
    speed = 1;
    wMax = 1;
    # do dStep1 here

    # control trying to min or max value function?
    uMode = 'min';
    # do dStep2 here

    ## Pack problem parameters

    # Define dynamic system
    # obj = DubinsCar(x, wMax, speed, dMax)
    dCar = DubinsCar(zeros(1, 3), wMax, speed); #do dStep3 here

    # Put grid and dynamic systems into schemeData
    scemaData = Bundle(dict(grid = g, dynSys = dCar, accuracy = 'high',
                            uMode = uMode))
    #do dStep4 here

    ## additive random noise
    #do Step8 here
    #HJIextraArgs.addGaussianNoiseStandardDeviation = [0; 0; 0.5];
    # Try other noise coefficients, like:
    #    [0.2; 0; 0]; # Noise on X state
    #    [0.2,0,0;0,0.2,0;0,0,0.5]; # Independent noise on all states
    #    [0.2;0.2;0.5]; # Coupled noise on all states
    #    {zeros(size(g.xs{1})); zeros(size(g.xs{1})); (g.xs{1}+g.xs{2})/20}; # State-dependent noise

    ## If you have obstacles, compute them here

    ## Compute value function
    HJIextraArgs = Bundle(dict())
    HJIextraArgs.visualize = Bundle(dict())

    #HJIextraArgs.visualize = True; #show plot
    HJIextraArgs.visualize.valueSet = 1;
    HJIextraArgs.visualize.initialValueSet = 1;
    HJIextraArgs.visualize.figNum = 1; #set figure number
    HJIextraArgs.visualize.deleteLastPlot = True; #delete previous plot as you update

    # uncomment if you want to see a 2D slice
    # HJIextraArgs.visualize.plotData = Bundle(dict())
    #HJIextraArgs.visualize.plotData.plotDims = [1 1 0]; #plot x, y
    #HJIextraArgs.visualize.plotData.projpt = [0]; #project at theta = 0
    #HJIextraArgs.visualize.viewAngle = [0,90]; # view 2D

    #[data, tau, extraOuts] = ...
    # HJIPDE_solve(data0, tau, schemeData, minWith, extraArgs)
    [data, tau2, _] = HJIPDE_solve(data0, tau, schemeData, 'none', HJIextraArgs);

    ## Compute optimal trajectory from some initial state
    if compTraj:

        #set the initial state
        xinit = expand(np.array((2, 2, -pi)), 0)

        #check if this initial state is in the BRS/BRT
        #value = eval_u(g, data, x)
        value = eval_u(g,data[:,:,:,-1],xinit);

        if value <= 0: #if initial state is in BRS/BRT
            # find optimal trajectory

            dCar.x = xinit; #set initial state of the dubins car

            TrajextraArgs = Bundle(dict(
                                uMode = uMode, #set if control wants to min or max
                                dMode = 'max',
                                visualize = True, #show plot
                                fig_num = 2, #figure number
                                #we want to see the first two dimensions (x and y)
                                projDim = expand(np.array((1 1 0)), 0)
                            ))


            #flip data time points so we start from the beginning of time
            dataTraj = np.flip(data,4); # I hope this is correct

            # [traj, traj_tau] = ...
            # computeOptTraj(g, data, tau, dynSys, extraArgs)
            [traj, traj_tau] = ...
              computeOptTraj(g, dataTraj, tau2, dCar, TrajextraArgs);

            # fig = plt.gcf()
            # plt.clf()
            # h = visSetIm(g, data[:,:,:,-1]);
            # # h.FaceAlpha = .3;
            #
            # ax = fig.add_subplot(projection='3d')
            # ax.scatter(xinit[1], xinit[2], xinit[3]);
            # # s.SizeData = 70;
            # ax.set_title('The reachable set at the end and x_init')
            #
            # #plot traj
            # # figure(4)
            # ax2 = fig.add_subplot(1, 1, 1)
            # ax2.plot(traj[0,:], traj[1,:])
            # ax2.set_xlim(left=-5, right=5)
            # ax2.set_ylim(left=-5, right=5)
            # add the target set to that
            g2D, data2D = proj(g, data0, [0, 0, 1]);
            # visSetIm(g2D, data2D, 'green');
            # ax2.set_title('2D projection of the trajectory & target set')
            # hold off
        else:
            logger.fatal(f'Initial state is not in the BRS/BRT! It have a value of {value}')
        return g2D, data2D


Variable         Type        Data/Info
--------------------------------------
Bundle           type        <class 'gen_utils.Bundle'>
abspath          function    <function abspath at 0x7f3ce0f7b680>
cell             function    <function cell at 0x7f3cdf60acb0>
createGrid       function    <function createGrid at 0x7f3cdc183560>
expand           function    <function expand at 0x7f3cdf60a9e0>
isColumnLength   function    <function isColumnLength at 0x7f3cdf60ac20>
iscell           function    <function iscell at 0x7f3cdf60ad40>
isscalar         function    <function isscalar at 0x7f3cdf60add0>
join             function    <function join at 0x7f3ce0f7af80>
ndims            function    <function ndims at 0x7f3cdf60ab90>
np               module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
numDims          function    <function numDims at 0x7f3c98a64f80>
ones             function    <function ones at 0x7f3cdf60aa70>
plt              module      <module 'matplotlib.pyplo<..

In [8]:
data= Bundle(dict(a=1, b=2, c=3))
data.a

1

In [24]:
help(np.nonzero)

Help on function nonzero in module numpy:

nonzero(a)
    Return the indices of the elements that are non-zero.
    
    Returns a tuple of arrays, one for each dimension of `a`,
    containing the indices of the non-zero elements in that
    dimension. The values in `a` are always tested and returned in
    row-major, C-style order.
    
    To group the indices by element, rather than dimension, use `argwhere`,
    which returns a row for each non-zero element.
    
    .. note::
    
       When called on a zero-d array or scalar, ``nonzero(a)`` is treated
       as ``nonzero(atleast1d(a))``.
    
       .. deprecated:: 1.17.0
    
          Use `atleast1d` explicitly if this behavior is deliberate.
    
    Parameters
    ----------
    a : array_like
        Input array.
    
    Returns
    -------
    tuple_of_arrays : tuple
        Indices of elements that are non-zero.
    
    See Also
    --------
    flatnonzero :
        Return indices that are non-zero in the flattened ve

In [20]:
a = np.arange(8).reshape(4,2)
print(a)
a[a>0]

[[0 1]
 [2 3]
 [4 5]
 [6 7]]


array([1, 2, 3, 4, 5, 6, 7])

In [32]:
print(a)
idces = np.nonzero(a==5)
print(idces)
a[idces]

[[0 1]
 [2 3]
 [4 5]
 [6 7]]
(array([2]), array([1]))


array([5])

In [33]:
len(np.random.randn(2,8))

2

In [149]:
data = np.random.randn(4,4)
print(data)
for i in range(4):
#     col = [[':'] for i in range(4)]
#     col[i] = 1
#     print('col[:]: ', col[:])
#     print('data[col[:]] ', data[col[:]])
    data = np.concatenate((data, np.expand_dims(data[i], 0)), 0)
    #print(i, ':',  data)
    

[[ 0.06244107  0.69529362  0.6326224   0.58253316]
 [-1.21416698  1.45355482 -0.51653236  0.67805246]
 [-1.78050884  0.49770683 -1.58653607  0.54979628]
 [-0.84867089 -0.552695   -0.45479385  1.47591256]]


In [150]:
data.shape

(8, 4)